<a href="https://colab.research.google.com/github/AleAguirreM/Proyecto-ACCIDENTALIDAD-VIAL/blob/main/Proyecto_Movilidad_Vial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import sqlite3 as sql
import plotly.graph_objs as go
import plotly.express as px
import sys
from mlxtend.preprocessing import TransactionEncoder

In [133]:
# Cargar datos historicosr
ruta_archivo='https://raw.githubusercontent.com/AleAguirreM/Proyecto-ACCIDENTALIDAD-VIAL/main/AMVA_Accidentalidad_20191022_2.csv'

In [134]:

df = pd.read_csv(ruta_archivo, sep=';',encoding='latin1')




In [135]:
df


,COD_MUNICIPIO,MUNICIPIO,FECHA,HORA,DÍA DE LA SEMANA,CLASE,DIRECCIÓN,GRAVEDAÑOSSADAÑOSS,BARRIO,COMUNA,DISEÑO
0,79.0,BARBOSA,03/01/2015,01:00:00 PM,SABADO,CHOQUE,VIA BELLO HATILLO KM16+200,HERIDOS,EL HATILLO,SIN INFORMACIÓN,TRAMO DE VIA
1,79.0,BARBOSA,07/01/2015,03:00:00 PM,MIERCOLES,CHOQUE,HATILLO-BELLO KM 18+00,HERIDOS,PARQUE PRINCIPAL,SIN INFORMACIÓN,TRAMO DE VIA
2,79.0,BARBOSA,04/01/2015,04:01:00 AM,DOMINGO,CHOQUE,KM 16+395 METROS BARBOSA-HATILLO,HERIDOS,VIA EL HATILLO-CISNEROS,SIN INFORMACIÓN,TRAMO DE VIA
3,79.0,BARBOSA,05/01/2015,12:00:00 PM,LUNES,CHOQUE,BELLO- DON MATIAS KM 9+600,HERIDOS,EL HATILLO,SIN INFORMACIÓN,TRAMO DE VIA
4,79.0,BARBOSA,11/01/2015,01:30:00 PM,DOMINGO,CHOQUE,LOS ABUELOS,DAÑOS,PARQUE PRINCIPAL,SIN INFORMACIÓN,TRAMO DE VIA
...,...,...,...,...,...,...,...,...,...,...,...
203445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203450 entries, 0 to 203449
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   COD_MUNICIPIO       203435 non-null  float64
 1   MUNICIPIO           203435 non-null  object 
 2   FECHA               203435 non-null  object 
 3   HORA                203435 non-null  object 
 4   DÍA DE LA SEMANA    203435 non-null  object 
 5   CLASE               203435 non-null  object 
 6   DIRECCIÓN           203435 non-null  object 
 7   GRAVEDAÑOSSADAÑOSS  203435 non-null  object 
 8   BARRIO              203385 non-null  object 
 9   COMUNA              203435 non-null  object 
 10  DISEÑO              203435 non-null  object 
dtypes: float64(1), object(10)
memory usage: 17.1+ MB


In [137]:
print(df.columns)

Index(['COD_MUNICIPIO', 'MUNICIPIO', 'FECHA', 'HORA', 'DÍA DE LA SEMANA',
       'CLASE', 'DIRECCIÓN', 'GRAVEDAÑOSSADAÑOSS', 'BARRIO', 'COMUNA',
       'DISEÑO'],
      dtype='object')


In [138]:
df = df.rename(columns={
    'GRAVEDAÑOSSADAÑOSS': 'GRAVEDAD_DAÑOS',
    'DÍA DE LA SEMANA': 'DIA_SEMANA'
})

In [139]:
print("\nValores nulos por columna:\n")
print(df.isnull().sum())


Valores nulos por columna:

COD_MUNICIPIO     15
MUNICIPIO         15
FECHA             15
HORA              15
DIA_SEMANA        15
CLASE             15
DIRECCIÓN         15
GRAVEDAD_DAÑOS    15
BARRIO            65
COMUNA            15
DISEÑO            15
dtype: int64


In [140]:
df = df.dropna()

In [141]:
#df = df.drop_duplicates()

In [142]:

# Convertir FECHA a formato datetime
if 'FECHA' in df.columns:
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    print(f"✓ Fecha convertida a formato datetime")

# Limpiar y estandarizar HORA
if 'HORA' in df.columns:
    # Primero convertir a string y limpiar
    df['HORA'] = df['HORA'].astype(str).str.strip()

    # Función para convertir hora
    def convertir_hora(hora_str):
        try:
            # Si ya es un objeto tiempo o datetime
            if isinstance(hora_str, (datetime, pd.Timestamp)):
                return hora_str.time()

            # Si está en formato HH:MM
            if ':' in hora_str and len(hora_str) <= 5:
                return pd.to_datetime(hora_str, format='%H:%M').time()

            # Si tiene segundos u otro formato
            return pd.to_datetime(hora_str, errors='coerce').time()

        except:
            return None

    df['HORA_LIMPIA'] = df['HORA'].apply(convertir_hora)
    df['HORA_STR'] = df['HORA_LIMPIA'].apply(lambda x: x.strftime('%H:%M') if x else None)
    print(f"✓ Hora limpiada y estandarizada")

✓ Fecha convertida a formato datetime


/tmp/ipython-input-3637690702.py:23: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(hora_str, errors='coerce').time()


✓ Hora limpiada y estandarizada


In [143]:
# Convertir FECHA a formato datetime
if 'FECHA' in df.columns:
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    print(f"✓ Fecha convertida a formato datetime")

# Limpiar y estandarizar HORA
if 'HORA' in df.columns:
    # Primero convertir a string y limpiar
    df['HORA'] = df['HORA'].astype(str).str.strip()

    # Función para convertir hora
    def convertir_hora(hora_str):
        try:
            # Si ya es un objeto tiempo o datetime
            if isinstance(hora_str, (datetime, pd.Timestamp)):
                return hora_str.time()

            # Si está en formato HH:MM
            if ':' in hora_str and len(hora_str) <= 5:
                return pd.to_datetime(hora_str, format='%H:%M').time()

            # Si tiene segundos u otro formato
            return pd.to_datetime(hora_str, errors='coerce').time()

        except:
            return None

    df['HORA_LIMPIA'] = df['HORA'].apply(convertir_hora)
    df['HORA_STR'] = df['HORA_LIMPIA'].apply(lambda x: x.strftime('%H:%M') if x else None)
    print(f"✓ Hora limpiada y estandarizada")

# Estandarizar texto en columnas categóricas
if 'DÍA_SEMANA' in df.columns:
    df['DÍA_SEMANA'] = df['DÍA_SEMANA'].str.upper().str.strip()

if 'CLASE' in df.columns:
    df['CLASE'] = df['CLASE'].str.upper().str.strip()

if 'GRAVEDAD_DAÑOS' in df.columns:
    df['GRAVEDAD_DAÑOS'] = df['GRAVEDAD_DAÑOS'].str.upper().str.strip()

if 'BARRIO' in df.columns:
    df['BARRIO'] = df['BARRIO'].str.upper().str.strip()

if 'COMUNA' in df.columns:
    df['COMUNA'] = df['COMUNA'].str.upper().str.strip()

# 4. CREAR VARIABLES DERIVADAS
print("\n4. Creando variables derivadas...")

if 'FECHA' in df.columns:
    df['MES'] = df['FECHA'].dt.month
    df['DIA_SEMANA_NUM'] = df['FECHA'].dt.dayofweek
    df['DIA_MES'] = df['FECHA'].dt.day

if 'HORA_LIMPIA' in df.columns:
    df['HORA_NUM'] = df['HORA_LIMPIA'].apply(lambda x: x.hour if x else None)
    df['FRANJA_HORARIA'] = df['HORA_NUM'].apply(lambda x:
        'MADRUGADA (0-6)' if 0 <= x < 6 else
        'MAÑANA (6-12)' if 6 <= x < 12 else
        'TARDE (12-18)' if 12 <= x < 18 else
        'NOCHE (18-24)' if 18 <= x <= 24 else 'DESCONOCIDO'
    )



✓ Fecha convertida a formato datetime


/tmp/ipython-input-3322120204.py:23: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(hora_str, errors='coerce').time()


✓ Hora limpiada y estandarizada

4. Creando variables derivadas...


In [144]:
def analisis_exploratorio(df):
    """
    Análisis exploratorio básico de los datos limpios
    """

    # 1. Estadísticas básicas
    print("1. ESTADÍSTICAS BÁSICAS:")
    print(f"Total de accidentes: {len(df)}")
    print(f"Período analizado: {df['FECHA'].min().strftime('%d/%m/%Y')} - {df['FECHA'].max().strftime('%d/%m/%Y')}")

    # 2. Distribución por gravedad
    print("\n2. DISTRIBUCIÓN POR GRAVEDAD:")
    distrib_gravedad = df['GRAVEDAD_DAÑOS'].value_counts()
    for gravedad, cantidad in distrib_gravedad.items():
        porcentaje = (cantidad / len(df)) * 100
        print(f"  {gravedad}: {cantidad} ({porcentaje:.1f}%)")

    # 3. Tipos de accidente más comunes
    print("\n3. TIPOS DE ACCIDENTE MÁS COMUNES:")
    distrib_clase = df['CLASE'].value_counts().head(5)
    for clase, cantidad in distrib_clase.items():
        porcentaje = (cantidad / len(df)) * 100
        print(f"  {clase}: {cantidad} ({porcentaje:.1f}%)")

    # 4. Comunas con más accidentes
    print("\n4. COMUNAS CON MÁS ACCIDENTES:")
    distrib_comuna = df['COMUNA'].value_counts().head(5)
    for comuna, cantidad in distrib_comuna.items():
        porcentaje = (cantidad / len(df)) * 100
        print(f"  {comuna}: {cantidad} ({porcentaje:.1f}%)")

    # 5. Distribución por franja horaria
    print("\n5. DISTRIBUCIÓN POR FRANJA HORARIA:")
    if 'FRANJA_HORARIA' in df.columns:
        distrib_horaria = df['FRANJA_HORARIA'].value_counts()
        for franja, cantidad in distrib_horaria.items():
            porcentaje = (cantidad / len(df)) * 100
            print(f"  {franja}: {cantidad} ({porcentaje:.1f}%)")


In [145]:
analisis_exploratorio(df)


1. ESTADÍSTICAS BÁSICAS:
Total de accidentes: 203385
Período analizado: 01/01/2015 - 31/12/2018

2. DISTRIBUCIÓN POR GRAVEDAD:
  HERIDOS: 101378 (49.8%)
  DAÑOS: 100927 (49.6%)
  MUERTOS: 1080 (0.5%)

3. TIPOS DE ACCIDENTE MÁS COMUNES:
  CHOQUE: 146126 (71.8%)
  ATROPELLO: 17280 (8.5%)
  OTRO: 15983 (7.9%)
  CAIDA OCUPANTE: 15641 (7.7%)
  VOLCAMIENTO: 8323 (4.1%)

4. COMUNAS CON MÁS ACCIDENTES:
  SIN INFORMACIÓN: 59727 (29.4%)
  LA CANDELARIA: 29947 (14.7%)
  LAURELES ESTADIO: 16631 (8.2%)
  CASTILLA: 14500 (7.1%)
  EL POBLADO: 11889 (5.8%)

5. DISTRIBUCIÓN POR FRANJA HORARIA:
  TARDE (12-18): 72447 (35.6%)
  MAÑANA (6-12): 68862 (33.9%)
  NOCHE (18-24): 41652 (20.5%)
  MADRUGADA (0-6): 20423 (10.0%)
  DESCONOCIDO: 1 (0.0%)


In [146]:
def ejecutar_sql (nombre_archivo, cur):
    sql_file=open(nombre_archivo)
    sql_as_string=sql_file.read()
    sql_file.close
    cur.executescript(sql_as_string)



In [147]:
df

,COD_MUNICIPIO,MUNICIPIO,FECHA,HORA,DIA_SEMANA,CLASE,DIRECCIÓN,GRAVEDAD_DAÑOS,BARRIO,COMUNA,DISEÑO,HORA_LIMPIA,HORA_STR,MES,DIA_SEMANA_NUM,DIA_MES,HORA_NUM,FRANJA_HORARIA
0,79.0,BARBOSA,2015-01-03,01:00:00 PM,SABADO,CHOQUE,VIA BELLO HATILLO KM16+200,HERIDOS,EL HATILLO,SIN INFORMACIÓN,TRAMO DE VIA,13:00:00,13:00,1.0,5.0,3.0,13.0,TARDE (12-18)
1,79.0,BARBOSA,2015-01-07,03:00:00 PM,MIERCOLES,CHOQUE,HATILLO-BELLO KM 18+00,HERIDOS,PARQUE PRINCIPAL,SIN INFORMACIÓN,TRAMO DE VIA,15:00:00,15:00,1.0,2.0,7.0,15.0,TARDE (12-18)
2,79.0,BARBOSA,2015-01-04,04:01:00 AM,DOMINGO,CHOQUE,KM 16+395 METROS BARBOSA-HATILLO,HERIDOS,VIA EL HATILLO-CISNEROS,SIN INFORMACIÓN,TRAMO DE VIA,04:01:00,04:01,1.0,6.0,4.0,4.0,MADRUGADA (0-6)
3,79.0,BARBOSA,2015-01-05,12:00:00 PM,LUNES,CHOQUE,BELLO- DON MATIAS KM 9+600,HERIDOS,EL HATILLO,SIN INFORMACIÓN,TRAMO DE VIA,12:00:00,12:00,1.0,0.0,5.0,12.0,TARDE (12-18)
4,79.0,BARBOSA,2015-01-11,01:30:00 PM,DOMINGO,CHOQUE,LOS ABUELOS,DAÑOS,PARQUE PRINCIPAL,SIN INFORMACIÓN,TRAMO DE VIA,13:30:00,13:30,1.0,6.0,11.0,13.0,TARDE (12-18)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203430,631.0,SABANETA,2018-08-30,18:45:00,JUEVES,CHOQUE,CL 77 Sur CR 45,HERIDOS,SIN INFORMACIÓN,SIN INFORMACIÓN,SIN INFORMACIÓN,18:45:00,18:45,8.0,3.0,30.0,18.0,NOCHE (18-24)
203431,631.0,SABANETA,2018-08-30,18:45:00,JUEVES,CHOQUE,CL 77 Sur CR 45,HERIDOS,SIN INFORMACIÓN,SIN INFORMACIÓN,SIN INFORMACIÓN,18:45:00,18:45,8.0,3.0,30.0,18.0,NOCHE (18-24)
203432,631.0,SABANETA,2018-08-31,17:00:00,VIERNES,CAIDA OCUPANTE,CL 52 Sur CR 44,HERIDOS,SIN INFORMACIÓN,SIN INFORMACIÓN,SIN INFORMACIÓN,17:00:00,17:00,8.0,4.0,31.0,17.0,TARDE (12-18)
203433,631.0,SABANETA,2018-08-31,18:10:00,VIERNES,VOLCAMIENTO,CR 49 CL 51 Sur,HERIDOS,SIN INFORMACIÓN,SIN INFORMACIÓN,SIN INFORMACIÓN,18:10:00,18:10,8.0,4.0,31.0,18.0,NOCHE (18-24)


In [148]:
sys.path.append('df.db')

In [149]:
conn = sql.connect("df.db")

In [150]:
# 3️⃣ Guardar el DataFrame en una tabla de SQLite
df.to_sql('mi_tabla', conn, if_exists='replace', index=False)


203385

In [151]:
curr = conn.cursor()


In [152]:
curr.execute("SELECT name FROM sqlite_master WHERE type = 'table'")

In [153]:
curr.fetchall()

[('mi_tabla',)]

In [154]:
pd.read_sql("select * from mi_tabla", conn)



,COD_MUNICIPIO,MUNICIPIO,FECHA,HORA,DIA_SEMANA,CLASE,DIRECCIÓN,GRAVEDAD_DAÑOS,BARRIO,COMUNA,DISEÑO,HORA_LIMPIA,HORA_STR,MES,DIA_SEMANA_NUM,DIA_MES,HORA_NUM,FRANJA_HORARIA
0,79.0,BARBOSA,2015-01-03 00:00:00,01:00:00 PM,SABADO,CHOQUE,VIA BELLO HATILLO KM16+200,HERIDOS,EL HATILLO,SIN INFORMACIÓN,TRAMO DE VIA,13:00:00.000000,13:00,1.0,5.0,3.0,13.0,TARDE (12-18)
1,79.0,BARBOSA,2015-01-07 00:00:00,03:00:00 PM,MIERCOLES,CHOQUE,HATILLO-BELLO KM 18+00,HERIDOS,PARQUE PRINCIPAL,SIN INFORMACIÓN,TRAMO DE VIA,15:00:00.000000,15:00,1.0,2.0,7.0,15.0,TARDE (12-18)
2,79.0,BARBOSA,2015-01-04 00:00:00,04:01:00 AM,DOMINGO,CHOQUE,KM 16+395 METROS BARBOSA-HATILLO,HERIDOS,VIA EL HATILLO-CISNEROS,SIN INFORMACIÓN,TRAMO DE VIA,04:01:00.000000,04:01,1.0,6.0,4.0,4.0,MADRUGADA (0-6)
3,79.0,BARBOSA,2015-01-05 00:00:00,12:00:00 PM,LUNES,CHOQUE,BELLO- DON MATIAS KM 9+600,HERIDOS,EL HATILLO,SIN INFORMACIÓN,TRAMO DE VIA,12:00:00.000000,12:00,1.0,0.0,5.0,12.0,TARDE (12-18)
4,79.0,BARBOSA,2015-01-11 00:00:00,01:30:00 PM,DOMINGO,CHOQUE,LOS ABUELOS,DAÑOS,PARQUE PRINCIPAL,SIN INFORMACIÓN,TRAMO DE VIA,13:30:00.000000,13:30,1.0,6.0,11.0,13.0,TARDE (12-18)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203380,631.0,SABANETA,2018-08-30 00:00:00,18:45:00,JUEVES,CHOQUE,CL 77 Sur CR 45,HERIDOS,SIN INFORMACIÓN,SIN INFORMACIÓN,SIN INFORMACIÓN,18:45:00.000000,18:45,8.0,3.0,30.0,18.0,NOCHE (18-24)
203381,631.0,SABANETA,2018-08-30 00:00:00,18:45:00,JUEVES,CHOQUE,CL 77 Sur CR 45,HERIDOS,SIN INFORMACIÓN,SIN INFORMACIÓN,SIN INFORMACIÓN,18:45:00.000000,18:45,8.0,3.0,30.0,18.0,NOCHE (18-24)
203382,631.0,SABANETA,2018-08-31 00:00:00,17:00:00,VIERNES,CAIDA OCUPANTE,CL 52 Sur CR 44,HERIDOS,SIN INFORMACIÓN,SIN INFORMACIÓN,SIN INFORMACIÓN,17:00:00.000000,17:00,8.0,4.0,31.0,17.0,TARDE (12-18)
203383,631.0,SABANETA,2018-08-31 00:00:00,18:10:00,VIERNES,VOLCAMIENTO,CR 49 CL 51 Sur,HERIDOS,SIN INFORMACIÓN,SIN INFORMACIÓN,SIN INFORMACIÓN,18:10:00.000000,18:10,8.0,4.0,31.0,18.0,NOCHE (18-24)


In [156]:
### excluimos los usuarios que tengan menos de 300 peliculas calificadas
accidentes_municipio =pd.read_sql(''' select "MUNICIPIO" as MUNICIPIO,
                         count(*) as cnt_mun
                         from mi_tabla
                         group by "MUNICIPIO"
                         order by cnt_mun desc
                         ''',conn )
accidentes_municipio

,MUNICIPIO,cnt_mun
0,MEDELLÍN,143800
1,ENVIGADO,19623
2,ITAGUI,17477
3,SABANETA,8875
4,BELLO,7614
5,BARBOSA,3237
6,CALDAS,2759


In [160]:
##Graficamos para observar la distribuciòn con la restricciòn
fig  = px.histogram(accidentes_municipio, x= 'MUNICIPIO', y = 'cnt_mun', title= 'HIST RANKING ACCIDENTES POR MUNICIPIO')
fig.show()

In [162]:
fig = px.bar(
    accidentes_municipio.sort_values('cnt_mun', ascending=False),  # orden descendente
    x='MUNICIPIO',
    y='cnt_mun',
    title='🚦 Ranking de Accidentes por Municipio',
    color='cnt_mun',  # color según cantidad
    color_continuous_scale='Rainbow',  # escala cálida
    text='cnt_mun'  # mostrar número sobre cada barra
)

# Ajustes estéticos
fig.update_traces(
    texttemplate='%{text:,}',  # separador de miles
    textposition='outside'
)

fig.update_layout(
    title_font=dict(size=22, family='Arial Black', color='#333'),
    xaxis_title='Municipio',
    yaxis_title='Número de Accidentes',
    xaxis=dict(showgrid=False, tickangle=-45),
    yaxis=dict(showgrid=True, gridcolor='rgba(200,200,200,0.3)'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    coloraxis_colorbar=dict(title='Accidentes'),
    margin=dict(t=90, b=120, l=70, r=40)
)

fig.show()